Als erstes muss für jeder Trial Manuell die Header entfernt werden, dass die Spalten danach z.B LASI:X, LASI:Y, LASI:Z heissen,
als Referenz kann D:/MA/08.07 Aufnahmen MA/Alle Koordinaten/squat_bounce/Vicon/squat_bounce_60fps_ohneheader.csv dienen

# Berechnung der Gelenksmittelpunkte:
Hüfte wird mit Bell et al. Methode (Tylkowski mit angepassten Werten) berechnet. 
Die übrigen mit dem Mittelwert der lateralen und meidalen oder posterior&anterioren Markern.
Es werden alle Dateien im Input folder genommen und die neuen Dateilen als "dateiname"_mittelwert.csv im Ouput-Folder gespeichert

In [1]:
import pandas as pd
import numpy as np
import os
import glob

# Funktion zur Mittelwertberechnung zwischen zwei Spalten
def calculate_mean(df, col1, col2, new_col):
    df[new_col] = df[[col1, col2]].apply(
        lambda row: np.nan if row.isnull().any() else row.mean(), axis=1
    )
    return df

# Verzeichnis mit den Input-Dateien
input_folder = r'E:\Datenerhebung MA\Vicon_csv\ohneheader\MAN262\x'
output_folder = r'E:\Datenerhebung MA\Vicon_csv\mittelwert\MAN262'

# Stelle sicher, dass das Ausgabe-Verzeichnis existiert
os.makedirs(output_folder, exist_ok=True)

# Hole alle CSV-Dateien im Input-Ordner
files = glob.glob(os.path.join(input_folder, '*.csv'))

# Schleife über alle Dateien
for file_path in files:
    print(f"Bearbeite Datei: {file_path}")
    
    # Datei einlesen
    data = pd.read_csv(file_path)
    
    # LANK (Left Ankle)
    data = calculate_mean(data, 'LANKMED:X', 'LANKLAT:X', 'LANK:X')
    data = calculate_mean(data, 'LANKMED:Y', 'LANKLAT:Y', 'LANK:Y')
    data = calculate_mean(data, 'LANKMED:Z', 'LANKLAT:Z', 'LANK:Z')

    # RANK (Right Ankle)
    data = calculate_mean(data, 'RANKMED:X', 'RANKLAT:X', 'RANK:X')
    data = calculate_mean(data, 'RANKMED:Y', 'RANKLAT:Y', 'RANK:Y')
    data = calculate_mean(data, 'RANKMED:Z', 'RANKLAT:Z', 'RANK:Z')

    # LKNE (Left Knee)
    data = calculate_mean(data, 'LKNEMED:X', 'LKNELAT:X', 'LKNE:X')
    data = calculate_mean(data, 'LKNEMED:Y', 'LKNELAT:Y', 'LKNE:Y')
    data = calculate_mean(data, 'LKNEMED:Z', 'LKNELAT:Z', 'LKNE:Z')

    # RKNE (Right Knee)
    data = calculate_mean(data, 'RKNEMED:X', 'RKNELAT:X', 'RKNE:X')
    data = calculate_mean(data, 'RKNEMED:Y', 'RKNELAT:Y', 'RKNE:Y')
    data = calculate_mean(data, 'RKNEMED:Z', 'RKNELAT:Z', 'RKNE:Z')

    # RHIP und LHIP Berechnung mit Tylkowski
    data['Dist_LASI_RASI'] = np.sqrt((data['LASI:X'] - data['RASI:X'])**2 + 
                                     (data['LASI:Y'] - data['RASI:Y'])**2 + 
                                     (data['LASI:Z'] - data['RASI:Z'])**2)

    # Berechne den Mittelpunkt zwischen RASI und LASI (mid-ASIS)
    data['mid_ASIS_X'] = (data['RASI:X'] + data['LASI:X']) / 2
    data['mid_ASIS_Y'] = (data['RASI:Y'] + data['LASI:Y']) / 2
    data['mid_ASIS_Z'] = (data['RASI:Z'] + data['LASI:Z']) / 2
    
    # Berechne den Mittelpunkt zwischen RPSI und LPSI (mid-PSIS)
    data['mid_PSI_X'] = (data['RPSI:X'] + data['LPSI:X']) / 2
    data['mid_PSI_Y'] = (data['RPSI:Y'] + data['LPSI:Y']) / 2
    data['mid_PSI_Z'] = (data['RPSI:Z'] + data['LPSI:Z']) / 2
    
    # Berechnung der Achsen
    Z_axis = np.array([data['RASI:X'] - data['LASI:X'], 
                       data['RASI:Y'] - data['LASI:Y'], 
                       data['RASI:Z'] - data['LASI:Z']]).T
    Z_axis_normalized = Z_axis / np.linalg.norm(Z_axis, axis=1)[:, np.newaxis]

    X_axis = np.array([data['mid_PSI_X'] - data['mid_ASIS_X'], 
                       data['mid_PSI_Y'] - data['mid_ASIS_Y'], 
                       data['mid_PSI_Z'] - data['mid_ASIS_Z']]).T
    X_axis_normalized = X_axis / np.linalg.norm(X_axis, axis=1)[:, np.newaxis]

    Y_axis_normalized = np.cross(X_axis_normalized, Z_axis_normalized)

    # Berechnung der Hüftgelenkpunkte (Hip Joint Centers, HJC)
    data['LHIP:X'] = data['LASI:X'] + 0.19 * data['Dist_LASI_RASI'] * X_axis_normalized[:, 0] - 0.30 * data['Dist_LASI_RASI'] * Y_axis_normalized[:, 0] + 0.14 * data['Dist_LASI_RASI'] * Z_axis_normalized[:, 0]
    data['LHIP:Y'] = data['LASI:Y'] + 0.19 * data['Dist_LASI_RASI'] * X_axis_normalized[:, 1] - 0.30 * data['Dist_LASI_RASI'] * Y_axis_normalized[:, 1] + 0.14 * data['Dist_LASI_RASI'] * Z_axis_normalized[:, 1]
    data['LHIP:Z'] = data['LASI:Z'] + 0.19 * data['Dist_LASI_RASI'] * X_axis_normalized[:, 2] - 0.30 * data['Dist_LASI_RASI'] * Y_axis_normalized[:, 2] + 0.14 * data['Dist_LASI_RASI'] * Z_axis_normalized[:, 2]

    data['RHIP:X'] = data['RASI:X'] + 0.19 * data['Dist_LASI_RASI'] * X_axis_normalized[:, 0] - 0.30 * data['Dist_LASI_RASI'] * Y_axis_normalized[:, 0] - 0.14 * data['Dist_LASI_RASI'] * Z_axis_normalized[:, 0]
    data['RHIP:Y'] = data['RASI:Y'] + 0.19 * data['Dist_LASI_RASI'] * X_axis_normalized[:, 1] - 0.30 * data['Dist_LASI_RASI'] * Y_axis_normalized[:, 1] - 0.14 * data['Dist_LASI_RASI'] * Z_axis_normalized[:, 1]
    data['RHIP:Z'] = data['RASI:Z'] + 0.19 * data['Dist_LASI_RASI'] * X_axis_normalized[:, 2] - 0.30 * data['Dist_LASI_RASI'] * Y_axis_normalized[:, 2] - 0.14 * data['Dist_LASI_RASI'] * Z_axis_normalized[:, 2]

    # HIPMIDDLE (Hip Middle)
    data['HIPMIDDLE:X'] = data[['LPSI:X', 'LASI:X', 'RPSI:X', 'RASI:X']].apply(
        lambda row: np.nan if row.isnull().any() else row.mean(), axis=1
    )
    data['HIPMIDDLE:Y'] = data[['LPSI:Y', 'LASI:Y', 'RPSI:Y', 'RASI:Y']].apply(
        lambda row: np.nan if row.isnull().any() else row.mean(), axis=1
    )
    data['HIPMIDDLE:Z'] = data[['LPSI:Z', 'LASI:Z', 'RPSI:Z', 'RASI:Z']].apply(
        lambda row: np.nan if row.isnull().any() else row.mean(), axis=1
    )
    
    # LSHO (Left Shoulder)
    data = calculate_mean(data, 'LSHOFRONT:X', 'LSHOBACK:X', 'LSHO:X')
    data = calculate_mean(data, 'LSHOFRONT:Y', 'LSHOBACK:Y', 'LSHO:Y')
    data = calculate_mean(data, 'LSHOFRONT:Z', 'LSHOBACK:Z', 'LSHO:Z')
    
    # RSHO (Right Shoulder)
    data = calculate_mean(data, 'RSHOFRONT:X', 'RSHOBACK:X', 'RSHO:X')
    data = calculate_mean(data, 'RSHOFRONT:Y', 'RSHOBACK:Y', 'RSHO:Y')
    data = calculate_mean(data, 'RSHOFRONT:Z', 'RSHOBACK:Z', 'RSHO:Z')
    
    # LELB (Left Elbow)
    data = calculate_mean(data, 'LELBLAT:X', 'LELBMED:X', 'LELB:X')
    data = calculate_mean(data, 'LELBLAT:Y', 'LELBMED:Y', 'LELB:Y')
    data = calculate_mean(data, 'LELBLAT:Z', 'LELBMED:Z', 'LELB:Z')
    
    # RELB (Right Elbow)
    data = calculate_mean(data, 'RELBLAT:X', 'RELBMED:X', 'RELB:X')
    data = calculate_mean(data, 'RELBLAT:Y', 'RELBMED:Y', 'RELB:Y')
    data = calculate_mean(data, 'RELBLAT:Z', 'RELBMED:Z', 'RELB:Z')
    
    # LWRIST (Left Wrist)
    data = calculate_mean(data, 'LWRA:X', 'LWRB:X', 'LWRIST:X')
    data = calculate_mean(data, 'LWRA:Y', 'LWRB:Y', 'LWRIST:Y')
    data = calculate_mean(data, 'LWRA:Z', 'LWRB:Z', 'LWRIST:Z')
    
    # RWRIST (Right Wrist)
    data = calculate_mean(data, 'RWRA:X', 'RWRB:X', 'RWRIST:X')
    data = calculate_mean(data, 'RWRA:Y', 'RWRB:Y', 'RWRIST:Y')
    data = calculate_mean(data, 'RWRA:Z', 'RWRB:Z', 'RWRIST:Z')
    
    # Reihenfolge der Spalten neu festlegen
    new_columns = ['Frame', 
                   'HIPMIDDLE:X', 'HIPMIDDLE:Y', 'HIPMIDDLE:Z',
                   'RHIP:X', 'RHIP:Y', 'RHIP:Z',
                   'RKNE:X', 'RKNE:Y', 'RKNE:Z',
                   'RANK:X', 'RANK:Y', 'RANK:Z',
                   'LHIP:X', 'LHIP:Y', 'LHIP:Z',
                   'LKNE:X', 'LKNE:Y', 'LKNE:Z',
                   'LANK:X', 'LANK:Y', 'LANK:Z',
                   'LSHO:X', 'LSHO:Y', 'LSHO:Z',
                   'LELB:X', 'LELB:Y', 'LELB:Z',
                   'LWRIST:X', 'LWRIST:Y', 'LWRIST:Z',
                   'RSHO:X', 'RSHO:Y', 'RSHO:Z',
                   'RELB:X', 'RELB:Y', 'RELB:Z',
                   'RWRIST:X', 'RWRIST:Y', 'RWRIST:Z']

    # Daten filtern und neue Reihenfolge anwenden
    data_filtered = data[new_columns]

    # Output-Dateipfad erstellen
    base_name = os.path.basename(file_path)  # Originaldateiname
    output_file_name = base_name.replace('_noheader.csv', '_mittelwert.csv')
    output_file_path = os.path.join(output_folder, output_file_name)

    # Gefilterte Daten speichern
    data_filtered.to_csv(output_file_path, index=False)
    print(f"Datei gespeichert: {output_file_path}")

print("Alle Dateien erfolgreich bearbeitet und gespeichert.")

Bearbeite Datei: E:\Datenerhebung MA\Vicon_csv\ohneheader\MAN262\x\09_ebs_noheader.csv
Datei gespeichert: E:\Datenerhebung MA\Vicon_csv\mittelwert\MAN262\09_ebs_mittelwert.csv
Alle Dateien erfolgreich bearbeitet und gespeichert.
